In [1]:
import cv2
import numpy as np

def blur_faces_video(video_path, output_path):
    try:
        # Load the pre-trained face detection model
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        # Open the video file
        video_capture = cv2.VideoCapture(video_path)

        if not video_capture.isOpened():
            print(f"Error: Could not open video file '{video_path}'.")
            return False

        # Get the frames per second (fps) of the video
        fps = video_capture.get(cv2.CAP_PROP_FPS)

        # Get the width and height of the video frames
        frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Define the codec (try different options if needed)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can try other codecs here
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

        # Initialize previous frame
        ret, prev_frame = video_capture.read()
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

        while True:
            ret, frame = video_capture.read()

            if not ret:
                break

            # Convert the current frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Detect faces in the grayscale frame
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (x, y, w, h) in faces:
                # Calculate optical flow
                p0 = np.array([[x + w // 2, y + h // 2]], dtype=np.float32).reshape(-1, 1, 2)
                p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, p0, None)

                # If optical flow is found, update face coordinates
                if st[0][0] == 1:
                    x_new, y_new = p1[0].ravel()
                    x, y = int(x_new - w // 2), int(y_new - h // 2)

                # Blur the face region
                face = frame[y:y+h, x:x+w]
                blurred_face = cv2.GaussianBlur(face, (99, 99), 0)
                frame[y:y+h, x:x+w] = blurred_face

            # Update previous frame and previous gray frame
            prev_gray = gray.copy()

            # Write the modified frame to the output video
            out.write(frame)

        # Release video capture and writer objects
        video_capture.release()
        out.release()

        print(f"Faces blurred successfully and video saved to '{output_path}'.")
        return True

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

# Example usage (replace with your video paths)
video_path = "people_video_1.mp4"
output_path = "blurred_faces_video.avi"

if blur_faces_video(video_path, output_path):
    print("Face blurring completed. You can view the modified video using a video player.")

Faces blurred successfully and video saved to 'blurred_faces_video.avi'.
Face blurring completed. You can view the modified video using a video player.
